In [2]:
import sqlite3
from pathlib import Path
import pandas as pd
import requests
import os

In [2]:
SCHEMA = '''
drop table if exists trip_updates_temp;
create table trip_updates_temp (
  id integer primary key autoincrement,
  update_timestamp DATETIME not null,
  timestamp DATETIME not null,
  trip_id int not null,
  stop_id int not null,
  delay int not null
);

drop table if exists trip_updates;
create table trip_updates (
  id integer primary key autoincrement,
  update_timestamp DATETIME not null,
  timestamp DATETIME not null,
  trip_id int not null,
  stop_id int not null,
  delay int not null
);

drop table if exists vehicle_positions_temp;
create table vehicle_positions_temp (
  id integer primary key autoincrement,
  update_timestamp int not null,
  trip_id int not null,
  route_id int not null,
  trip_start_time int not null,
  trip_start_date datetime not null,
  position_lat int not null,
  position_lon int not null,
  bearing int not null,
  speed int not null,
  current_status str not null,
  stop_id int not null,
  timestamp datetime not null,
  vehicle_id int not null
);

drop table if exists vehicle_positions;
create table vehicle_positions (
  id integer primary key autoincrement,
  update_timestamp int not null,
  trip_id int not null,
  route_id int not null,
  trip_start_time int not null,
  trip_start_date datetime not null,
  position_lat int not null,
  position_lon int not null,
  bearing int not null,
  speed int not null,
  current_status str not null,
  stop_id int not null,
  timestamp datetime not null,
  vehicle_id int not null
);
'''

### Prune DB (execute ONLY first time)

In [3]:
conn = sqlite3.connect('RIPTA.db')

In [4]:
conn.executescript(SCHEMA)

In [5]:
conn.commit()

In [6]:
conn.close()

### Clean temporary DB files

In [131]:
SCHEMA = '''
drop table if exists trip_updates_temp;
create table trip_updates_temp (
  id integer primary key autoincrement,
  update_timestamp DATETIME not null,
  timestamp DATETIME not null,
  trip_id int not null,
  stop_id int not null,
  delay int not null
);

drop table if exists vehicle_positions_temp;
create table vehicle_positions_temp (
  id integer primary key autoincrement,
  update_timestamp int not null,
  trip_id int not null,
  route_id int not null,
  start_time int not null,
  start_date datetime not null,
  position_lat int not null,
  position_lon int not null,
  bearing int not null,
  speed int not null,
  current_status str not null,
  stop_id int not null,
  timestamp datetime not null,
  vehicle_id int not null
);
'''

In [132]:
conn = sqlite3.connect('RIPTA.db')

In [133]:
conn.executescript(SCHEMA)

In [134]:
conn.commit()

In [135]:
conn.close()

### Pull Data

In [136]:
tripupdates_df = pd.DataFrame()
r = requests.get('http://realtime.ripta.com:81/api/tripupdates?format=json')
outl = []
for feedentity in r.json()['entity']:
    if feedentity['trip_update']['stop_time_update'] != None:
        for stoptimeupdate in feedentity['trip_update']['stop_time_update']: 
            if stoptimeupdate['arrival'] != None:
                if stoptimeupdate['arrival']['delay'] != 0:
                    outdict = {}
                    outdict['timestamp'] = feedentity['trip_update']['timestamp']
                    outdict['trip_id'] = feedentity['trip_update']['trip']['trip_id']
                    outdict['stop_id'] = stoptimeupdate['stop_id']
                    outdict['delay'] = stoptimeupdate['arrival']['delay']
                    outdict['update_timestamp'] = r.json()['header']['timestamp']
                    outl.append(outdict)
for key in outl[0].keys():
    tripupdates_df[key] = [d[key] for d in outl]

In [111]:
tripupdates_df.head()

,timestamp,trip_id,stop_id,delay,update_timestamp
0,1564421571,2854250,1915,300,1564421583
1,1564421571,2854250,1976,300,1564421583
2,1564421571,2854250,2060,300,1564421583
3,1564421571,2854250,56765,300,1564421583
4,1564421571,2854250,2010,300,1564421583


In [137]:
r = requests.get('http://realtime.ripta.com:81/api/vehiclepositions?format=json')
outl = []
vehiclepos_df = pd.DataFrame()
for feedentity in r.json()['entity']:
    outdict = {}
    outdict['update_timestamp'] = r.json()['header']['timestamp']
    outdict['trip_id'] = feedentity['vehicle']['trip']['trip_id']
    outdict['route_id'] = feedentity['vehicle']['trip']['route_id']
    outdict['start_time'] = feedentity['vehicle']['trip']['start_time']
    outdict['start_date'] = feedentity['vehicle']['trip']['start_date']
    outdict['position_lat'] = feedentity['vehicle']['position']['latitude']
    outdict['position_lon'] = feedentity['vehicle']['position']['longitude']
    outdict['bearing'] = feedentity['vehicle']['position']['bearing']
    outdict['speed'] = feedentity['vehicle']['position']['speed']
    outdict['current_status'] = feedentity['vehicle']['current_status']
    outdict['stop_id'] = feedentity['vehicle']['stop_id']
    outdict['timestamp'] = feedentity['vehicle']['timestamp']
    outdict['vehicle_id'] = feedentity['vehicle']['vehicle']['id']
    outl.append(outdict)
for key in outl[0].keys():
    vehiclepos_df[key] = [d[key] for d in outl]

In [113]:
vehiclepos_df.head()

,update_timestamp,trip_id,route_id,start_time,start_date,position_lat,position_lon,bearing,speed,current_status,stop_id,timestamp,vehicle_id
0,1564421636,2854250,63,13:15:00,20190729,41.506577,-71.303790,285.0,0.00000,2,1915,1564421560,549
1,1564421636,2854278,63,12:52:00,20190729,41.491333,-71.317460,30.0,0.00000,1,1382,1564421601,1001
2,1564421636,2854279,63,13:22:00,20190729,41.518867,-71.309940,360.0,5.81152,2,2025,1564421613,1002
3,1564421636,2854173,14,12:25:00,20190729,41.774944,-71.426765,15.0,29.95168,2,72170,1564421608,922
4,1564421636,2854186,14,12:50:00,20190729,41.629080,-71.462840,135.0,16.98752,2,3375,1564421613,548


### Insert Data

In [138]:
INSERT_STATEMENT_VP = """
INSERT INTO vehicle_positions_temp (
    update_timestamp, trip_id, route_id, start_time, start_date, position_lat, position_lon, bearing, speed, current_status, stop_id, timestamp, vehicle_id)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

In [139]:
INSERT_STATEMENT = """
INSERT INTO trip_updates_temp (
    update_timestamp, trip_id, stop_id, delay, timestamp)
    VALUES (?, ?, ?, ?, ?);
"""

In [140]:
conn = sqlite3.connect('RIPTA.db')

In [141]:
values = tripupdates_df[['update_timestamp', 'trip_id', 'stop_id', 'delay','timestamp']].values

In [142]:
conn.executemany(INSERT_STATEMENT,values)

In [143]:
values_vp = vehiclepos_df[['update_timestamp', 'trip_id', 'route_id','start_time','start_date','position_lat',
                           'position_lon','bearing','speed','current_status','stop_id','timestamp','vehicle_id']].values

In [144]:
conn.executemany(INSERT_STATEMENT_VP,values_vp)

In [145]:
conn.commit()

In [146]:
conn.close()

### Test for dupes

In [147]:
INSERT_STATEMENT = """
INSERT INTO trip_updates(update_timestamp, trip_id, stop_id, delay, timestamp)
SELECT update_timestamp, trip_id, stop_id, delay, timestamp FROM trip_updates_temp tut
WHERE 
NOT EXISTS (SELECT 1 FROM trip_updates tu
    WHERE tut.timestamp = tu.timestamp AND tut.trip_id = tu.trip_id AND tut.stop_id = tu.stop_id)
"""

In [148]:
INSERT_STATEMENT_VP = """
INSERT INTO vehicle_positions(update_timestamp, trip_id, route_id, trip_start_time, trip_start_date, position_lat, position_lon, bearing, speed, current_status, stop_id, timestamp, vehicle_id)
SELECT update_timestamp, trip_id, route_id, start_time, start_date, position_lat, position_lon, bearing, speed, current_status, stop_id, timestamp, vehicle_id FROM vehicle_positions_temp vpt
WHERE 
NOT EXISTS (SELECT 1 FROM vehicle_positions vp
    WHERE vpt.timestamp = vp.timestamp AND vpt.position_lat = vp.position_lat AND vpt.position_lon = vp.position_lon)
"""

In [149]:
conn = sqlite3.connect('RIPTA.db')

In [150]:
conn.execute(INSERT_STATEMENT)

In [151]:
conn.execute(INSERT_STATEMENT_VP)

In [152]:
conn.commit()

In [153]:
conn.close()

### Final Test

In [3]:
conn = sqlite3.connect('RIPTA.db')

In [4]:
cursor = conn.cursor()

In [5]:
cursor.execute('SELECT * FROM vehicle_positions LIMIT 5;')

In [6]:
cursor.fetchall()

[(1,
  1564775906,
  2854283,
  63,
  '15:22:00',
  20190802,
  41.49817,
  -71.30745,
  195,
  8.49376,
  2,
  1480,
  1564775822,
  1005),
 (2,
  1564775906,
  2854254,
  63,
  '15:15:00',
  20190802,
  41.52629,
  -71.2970047,
  15,
  10.28192,
  2,
  2415,
  1564775864,
  1002),
 (3,
  1564775906,
  2854255,
  63,
  '15:45:00',
  20190802,
  41.514,
  -71.30985,
  345,
  9.38784,
  2,
  56765,
  1564775870,
  543),
 (4,
  1564775906,
  2854178,
  14,
  '15:37:00',
  20190802,
  41.4935646,
  -71.43794,
  360,
  19.66976,
  2,
  1875,
  1564775851,
  547),
 (5,
  1564775906,
  2854170,
  14,
  '14:29:00',
  20190802,
  41.7626343,
  -71.4313354,
  15,
  9.83488,
  2,
  72170,
  1564775870,
  538)]

In [13]:
conn.close()